In [1]:
from langchain.chat_models import ChatOpenAI
import pandas as pd
from io import StringIO
import json
import os
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [2]:
with open("gen_evals.json") as f:
    raw_data = json.load(f)
data = []
i = 0
for obj in raw_data:
    if len(obj['output']) < 3:
        data.append(obj)
        i += 1
    if i >= 5:
        break
data

[{'input': ',681,0.30484943706856704,232,826,,NCW3ZQskMPW,0.46803768785657696,C9ylTU28ASu,,,,368,,,0.42645220808282225\n,,0.9953150876658492,,,578,919,,,0.43755675645325876,310,,jO3aBnZK5MXU4LoMi5bp8Gw,,pHYrcHvV9hVL7mQtzk,\nMEQDCWOnDV69zHWTkDDEvv,0.8147041991529793,0.7467567372196248,,,0.6538661437072572,,0.9130332390268918,,,0.465946718932984,,,,,\n0.5127628084107447,,,,,,,0.21557488173092643,,n12ORS8LoNsqD5Ln,,,965,,724,116.0\n,,,2zLkPnbL,PcO2KweqJHraL1yMt4BQwVr0sW,,,338,625,EuUFLTsuq,434,500,Y1yJ5Co2xhcvOQXiNuLxcQweOo1thxeYST,,m3asTDUrJEBcuurtpgNpsepTi3MIE7CenK7ZzzdSLhiMA,\n,49BRCMRPkybH4oKRqHe78SeqvQsjD8h5W6rme3ja,,JpqEeg,fRme9MagSpTQLJ9bBjVYbGKlo96aUZ9mR6TKcQTeqJN82LJd,,,1,2,3,4,5,6,abu1kO,,\n0.627059547992073,,0.9776791503293044,,,,A,692,65,Rzlasz5yaVQaYlwMP9FgAv1mTMPYW99rj7,0.23608245651973925,0.14838735235385392,0.7643396164334414,pA5JeaV2C2du9kaO1ZNTXnj5ZrxdqNgnKqBsfXucNSm2IN,,\nUDgOoqLyp7oYnqMcuIZt2gW9NVzebF,,492,,hTo8d,,B,0.7835542396460238,0.9554097311697083,0.7893359479479

In [3]:
from langsmith import Client
client = Client()
dataset_name = "Plates Dataset"

try:
    client.delete_dataset(dataset_name=dataset_name)
except:
    pass
# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=dataset_name, description="Plates Data.",
)
for d in data:
    # Each example must be unique and have inputs defined.
    # Outputs are optional
    client.create_example(
        inputs={"input": d["input"]},
        outputs={"output": json.dumps(d["output"])},
        dataset_id=dataset.id,
    )

In [4]:
from langchain.output_parsers.json import parse_json_markdown

In [5]:
import re
from typing import Any, Optional
import json

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.evaluation import StringEvaluator


class JSONEvaluator(StringEvaluator):

    @property
    def requires_input(self) -> bool:
        return True

    @property
    def requires_reference(self) -> bool:
        return True

    @property
    def evaluation_name(self) -> str:
        return "json_equivalence"

    def _evaluate_strings(
        self,
        prediction: str,
        input: Optional[str] = None,
        reference: Optional[str] = None,
        **kwargs: Any
    ) -> dict:
        try:
            pred = sorted(parse_json_markdown(prediction), key=lambda x: json.dumps(x))
            ref = sorted(parse_json_markdown(reference), key=lambda x: json.dumps(x))
            if pred == ref:
                return {"score": 1}
            else:
                return {"score": 0}
        except:
            return {"score": -1}

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.schema.output_parser import StrOutputParser

In [7]:
FIND_PLATES_PLATE_ORIENTATION = """Your job is to identify where in a dataframe plate data exists

Plate based data is always rectangular in nature, typically consisting of 24 (4x6), 96 (8x12), 384 (16x24), or 1536 (32 x 48) wells, and may be located anywhere within a dataset.
- There may be gaps in the data or surrounding rows and columns
- The plate data may be partial. This means that there may be columns or rows that have missing values. These still count and you should make sure to include them
- There may be multiple plates.
- Use heuristics, like looking for numbers and patterns consistent with plate dimensions.
- The sequence of the wells might also be a clue, as they are usually arranged in rows or columns and numbered sequentially.
- The header row for each plate (often consisting of monotonically increasing integers) should NOT be considered as the start of the plate.
- Likewise, the column with the column labels should NOT be considered as the start of the plate.
- Each chunk of data might contain multiple plates and there may be multiple chunks.

CONTENTS:

Use the `contents` field to describe the contents of the plate.
You can often look for text near the table to induce the `contents` field. You should use this text EXACTLY, do not add or change this in any way.

PARTIAL DATA:

Often times, you may have partial data. You can tell if you have partial data when a header row is longer than the plate data.
For example, let's look at this example:

```
,A,B,C,D,E
1,1,2,,,
2,3,4,,,
3,,,,,
4,,,,,
5,,,,,
```

In this situation, the `C, D, and E` Columns and the the `3,4,5` row are both missing. We can tell this by looking at the header columns and seeing a clear pattern.

In our response as to where the plate data exists, we should include those columns/rows. So the final answer should be:

```json
[{"row_start": 1, "row_end": 3, "col_start": 1, "col_end": 5}]
```

OUTPUT FORMATTING:

Use 0-indexing for row and column numbers in the final JSON output.

BEGIN!

Where are the plates in this data?

Be concise.

Produce your output as JSON. The format should be:
```json
[{"row_start": 25, "row_end": 30, "col_start": 1, "col_end": 12, "contents": "Entity ID"}]
```
""" 
USER_PLATE_ORIENTATION = """,,,,\nChemical,,1,2,3\n,A,SB123,SB124,SB125\n,B,SB123,SB124,SB126\n,C,,,\n,D,SB123,SB124,SB128\n,Control,1,2,3\n,A,Negative,Positive,Library\n,B,Negative,Positive,Library\n,C,Negative,Positive,Library\n,D,Negative,Positive,Library\nDox Concentration,,,,\n,1,2,3,\nA,0.005,0.005,0.005,\nB,0.005,0.005,0.005,\nC,0.005,0.005,0.005,\nD,0.005,0.005,0.005,\n,,,,\n,,,,\nPrimer,1,2,3,\nA,PR-001,PR-001,PR-001,\nB,PR-002,PR-002,PR-002,\nC,PR-003,PR-003,PR-003,\nD,PR-004,PR-004,PR-004,\n"""  # noqa: E501

ASSISTANT_PLATE_ORIENTATION = """
[{"row_start": 1, "row_end": 4, "col_start": 2, "col_end": 4, "contents": "Chemical"}, {"row_start": 6, "row_end": 9, "col_start": 2, "col_end": 4, "contents": "Control"},{"row_start": 12, "row_end": 15, "col_start": 1, "col_end": 3, "contents": "Dox Concentration"},{"row_start": 19, "row_end": 22, "col_start": 1, "col_end": 3, "contents": "Primer"}]
"""  # noqa: E501


In [8]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=FIND_PLATES_PLATE_ORIENTATION),
    #HumanMessage(content="This has an example with partial data: " + pd.read_csv(StringIO(USER_PLATE_ORIENTATION), header=None).to_markdown(), example=True),
    #AIMessage(content=ASSISTANT_PLATE_ORIENTATION, example=True),
    ("human", "{input}"),
])

In [10]:
llm = ChatOpenAI(temperature=0, model="gpt-4").with_fallbacks([ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k	")])

In [11]:
chain = {
    "input": lambda x: pd.read_csv(StringIO(x["input"]), header=None).to_markdown()
} | prompt | llm | StrOutputParser()

In [13]:
chain.invoke({"input": data[1]['input']})

'[{"row_start": 8, "row_end": 17, "col_start": 4, "col_end": 6, "contents": "Entity ID"}]'

In [14]:
data[1]['output']

[{'row_start': 9,
  'row_end': 12,
  'col_start': 5,
  'col_end': 10,
  'contents': 'Plate'}]

In [15]:
pd.read_csv(StringIO(data[1]['input']), header=None)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,NaN,NaN,NaN,NaN,0.9456945111197044,4TDAETYpsFzMQqsc,NaN,731,NaN,NaN,NaN,0.5922677590606381,0.7722707323746664,NaN,0.4924467993791929,0.017727784200053942,NaN,537,NaN,NaN
1,0.7715879472340539,NaN,SGXmVqSkMZcpoMtd0qWe7OuapwCpDix9QPFuOsud5o3fus7,NaN,206,439,824,105,NaN,NaN,kmp4oQ7ZgsPpdWEoHBW5bjYh5mv5ghI3,0.6982000163872422,NaN,NaN,0.43538405079926323,388,NaN,eyRUnQNBxqEIYXwka23MC87IAROc41M74sAjCpuPKZ4ExmKt,U2OYf,NaN
2,NaN,x3ceV7TQBJJw9p,NaN,NaN,NaN,NaN,NaN,643,Kdfc9RzbV7hEtasSSKWa5d,404,sV6AF3fPmG,3tUkFfYlN9aHmLjnG0AtsNYdbSJhDCw9,6M62o8raHaGHc51e0rvBUeNfJRt9FKbD0dwJnKdNhjRr,NaN,NaN,NaN,0.7533958838904378,EKmCu3nWrPzW4faDOfZ7YUMINXMGR8HINYi,0.2554710977715444,0.5626511156154925
3,0.5378705511467934,NaN,0.8054300997377667,LhkAcu32WPDQX2WpbF0vGPkrYkmV9TZTwwvpuxuRGR1Wl,NaN,juIM1R4il5rxd,NaN,0.20809391586596593,NaN,NaN,693,NaN,NaN,Ya5e4UiaO63E75ZInFURQoEODbvL1RAB4RyH,0.727981074107477,7Pj1i545hdrfmv,NaN,NaN,386,v3s0y4r32xYBW7sahlufY
4,0.1410210820055653,662,322,0.139220097114371,NaN,159,170,0.6264085478852918,NaN,0.26381889140479975,0.40002870800677304,0.5206003791259672,NaN,NaN,0.15396312919963873,SufWexRwfSHKR8VBt1FisW,733,NaN,NaN,NaN
5,NaN,821,y6xBb7BdzQKS3RtabVsbRdIbPtZ8gLC6bGcvH7UI0jp,A5nRaYbu7T3fLGLxUM,0.24676854213102561,NaN,NaN,7GOyR,0.2650450971628441,NaN,WOnyWu1Df,NaN,918,NaN,98,187,NaN,0.6642671130882266,Ze8Z6oLb626c,129
6,NaN,NaN,NaN,144,NaN,Sxisv,iwhDiwIHgA1JgLm3DXZFRWPP6p,NaN,NaN,491,NaN,0.9605193743331565,NaN,NaN,0.41141502520795015,NaN,vMJjagnpzTPqtgaSEtEl49bEAeEpoq0mUTeRU5vlkdp42iul,NaN,NaN,NaN
7,NaN,8rklHTUQjPE7K6SdlIq2Z3h1,NaN,344,NaN,0.7806638420311616,P7jkjHtGTX2q9Ptg2lK,NaN,0.36270007476251165,NaN,0.6012360030842847,NaN,NaN,NaN,0.7194968598592518,L3vly2nz3niDXwYvTLVZF9grcKI3HN9b,NaN,URPv7RPvHCkjgQtEzYKy8yyPkdFlLNBrClMP1s7VAL,NaN,NaN
8,834,NaN,763,vo2dEJsw76KI1D3Qo6sHLM,0.9336478692759924,1,2,3,4,5,6,NaN,NaN,Rumwgy2LKrkrUrQ4rr1I3sZnJxeL2DHUJEXKv40iuW,0.8997140006031908,NaN,NaN,878,723,NaN
9,eVeP3NevDv4HLEc2Pp89pP,LWHafc0WiY04KRBBm9kgmu8azkPyPuzC2atoe8W,NaN,6uQ10qFVkdEN9NtrnuO8q,A,88,0.7491572199532066,0.1108724016851994,0.40048157358008973,0.49165187474422034,0.08404115539727575,NaN,IWzPYqDiYGpI,0.8291259404132868,0.4603777663201366,NaN,NaN,0.6211447453282845,0.4104969945010284,NaN


In [140]:
from langchain.smith import RunEvalConfig, run_on_dataset

def create_chain():
    return chain

eval_config = RunEvalConfig(
  custom_evaluators=[JSONEvaluator()]
)
run_on_dataset(
    client=client,
    dataset_name=dataset_name,
    llm_or_chain_factory=create_chain,
    evaluation=eval_config,
    verbose=True,
)

View the evaluation results for project '3718647a85cc4ef2b04160a828897e12-RunnableSequence' at:
https://dev.langchain.plus/projects/p/6072ec77-8b24-4aa4-af98-b969e69e8366?eval=true


{'project_name': '3718647a85cc4ef2b04160a828897e12-RunnableSequence',
 'results': <Task pending name='Task-1419' coro=<_arun_on_examples() running at /Users/harrisonchase/.pyenv/versions/3.10.1/envs/plate-chain/lib/python3.10/site-packages/langchain/smith/evaluation/runner_utils.py:816>>}

In [71]:
chain.invoke({"input": USER_PLATE_ORIENTATION})

AIMessage(content='```json\n[\n  {"row_start": 2, "row_end": 5, "col_start": 2, "col_end": 4, "contents": "Chemical"},\n  {"row_start": 7, "row_end": 10, "col_start": 2, "col_end": 4, "contents": "Control"},\n  {"row_start": 13, "row_end": 16, "col_start": 1, "col_end": 3, "contents": "Dox Concentration"},\n  {"row_start": 20, "row_end": 23, "col_start": 1, "col_end": 3, "contents": "Primer"}\n]\n```', additional_kwargs={}, example=False)

In [35]:
from langchain.chains import create_extraction_chain

In [36]:
schema = {
    "properties": {
        "row_start": {"type": "integer"},
        "row_end": {"type": "integer"},
        "col_start": {"type": "integer"},
        "col_end": {"type": "integer"},
        "contents": {"type": "string"},
    },
    "required": ["row_start", "row_end", "col_start", "col_end", "contents"],
}

In [37]:
chain = create_extraction_chain(schema, ChatOpenAI(temperature=0, model="gpt-4"))

In [39]:
output = chain.invoke({"input": df.to_markdown()})

In [43]:
from kor import Object, Text, Number
from kor import create_extraction_chain as kor_extract


In [51]:
schema = Object(
    id="plate_data",
    description=(
        """Plate based data is always rectangular in nature, typically consisting of 24 (4x6), 96 (8x12), 384 (16x24), or 1536 (32 x 48) wells, and may be located anywhere within a dataset.
- There may be gaps in the data or surrounding rows and columns
- The plate data may be partial
- There may be multiple plates.
- Use heuristics, like looking for numbers and patterns consistent with plate dimensions.
- The sequence of the wells might also be a clue, as they are usually arranged in rows or columns and numbered sequentially.
- The header row for each plate (often consisting of monotonically increasing integers) should NOT be considered as the start of the plate.
- Likewise, the column with the column labels should NOT be considered as the start of the plate.

Each chunk of data might contain multiple plates and there may be multiple chunks.
Use the `contents` field to describe the contents of the plate.
You can often look for text near the table to induce the `contents` field. You should use this text EXACTLY, do not add or change this in any way.

Use 0-indexing for row and column numbers in the final output.
"""
    ),
    attributes=[
        Number(
            id="row_start",
            description="Start of plate data",
            examples=[],
        ),
        Number(
            id="row_end",
            description="End of plate data",
            examples=[],
        ),
        Number(
            id="col_start",
            description="Start of plate data",
            examples=[],
        ),
        Number(
            id="col_end",
            description="End of plate data",
            examples=[],
        ),
        Text(
            id="contents",
            description="contents of plate data",
            examples=[],
        ),
    ],
    many=True,
)

chain = kor_extract(ChatOpenAI(temperature=0, model="gpt-4"), schema, encoder_or_encoder_class='json')

In [52]:
chain.run(df.to_markdown())

{'data': {},
 'raw': '<json>[\n{\n"row_start": 2,\n"row_end": 5,\n"col_start": 2,\n"col_end": 4,\n"contents": "Chemical"\n},\n{\n"row_start": 7,\n"row_end": 10,\n"col_start": 2,\n"col_end": 4,\n"contents": "Control"\n},\n{\n"row_start": 13,\n"row_end": 16,\n"col_start": 1,\n"col_end": 3,\n"contents": "Dox Concentration"\n},\n{\n"row_start": 20,\n"row_end": 23,\n"col_start": 1,\n"col_end": 3,\n"contents": "Primer"\n}\n]</json>',
 'errors': [kor.exceptions.ParseError('The LLM has returned structured data which does not match the expected schema. Providing additional examples may help improve the parse.')],
 'validated_data': {}}

In [50]:
import json
json.loads('[\n{\n"row_start": 2,\n"row_end": 5,\n"col_start": 2,\n"col_end": 4,\n"contents": "Chemical"\n},\n{\n"row_start": 7,\n"row_end": 10,\n"col_start": 2,\n"col_end": 4,\n"contents": "Control"\n},\n{\n"row_start": 13,\n"row_end": 16,\n"col_start": 1,\n"col_end": 3,\n"contents": "Dox Concentration"\n},\n{\n"row_start": 20,\n"row_end": 23,\n"col_start": 1,\n"col_end": 3,\n"contents": "Primer"\n}\n]')

[{'row_start': 2,
  'row_end': 5,
  'col_start': 2,
  'col_end': 4,
  'contents': 'Chemical'},
 {'row_start': 7,
  'row_end': 10,
  'col_start': 2,
  'col_end': 4,
  'contents': 'Control'},
 {'row_start': 13,
  'row_end': 16,
  'col_start': 1,
  'col_end': 3,
  'contents': 'Dox Concentration'},
 {'row_start': 20,
  'row_end': 23,
  'col_start': 1,
  'col_end': 3,
  'contents': 'Primer'}]

In [202]:
from langchain.schema.runnable import RunnableMap

chain1 = RunnableMap({
    "foo": lambda x: x["bar"]
}) | ChatPromptTemplate.from_template("{foo}") | ChatOpenAI()
chain2 = ChatPromptTemplate.from_template("{foo}") | ChatOpenAI()

In [211]:
%%time
chain1.invoke({"bar": "hi"})

CPU times: user 56.1 ms, sys: 12.6 ms, total: 68.7 ms
Wall time: 2.21 s


AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False)

In [215]:
%%time
chain2.invoke({"foo": "hi"})

CPU times: user 36.4 ms, sys: 8.47 ms, total: 44.9 ms
Wall time: 934 ms


AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False)